In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from random import choice



import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import Adam

from  src.load import load_data

In [25]:
eeg1, eeg2, eeg3, eeg4, eeg5, eeg6, eeg7, pulse, x, y, z = load_data("../data/raw/X_train.h5")

In [6]:
Y = pd.read_csv("../data/raw/y_train.csv")
Y = Y.drop(columns=['index'])

In [26]:
def sync(array, order):
    return [duplicate for element in array for duplicate in [element] * order]

def normalize(array):
    return (array - np.mean(array)) / np.std(array)

In [28]:
def make_sample(indices):
    samples = np.zeros([len(indices), 1500])
    for i, idx in enumerate(indices):
          samples[([normalize(eeg1.iloc[idx, :]), 
              normalize(eeg2.iloc[idx, :]), 
              normalize(eeg3.iloc[idx, :]), 
              normalize(eeg4.iloc[idx, :]), 
              normalize(eeg5.iloc[idx, :]), 
              normalize(eeg6.iloc[idx, :]), 
              normalize(eeg7.iloc[idx, :]),
              normalize(sync(pulse.iloc[idx, :], 5)),
              normalize(sync(x.iloc[idx, :], 5)),
              normalize(sync(y.iloc[idx, :], 5)),
              normalize(sync(z.iloc[idx, :], 5))])
    return samples

In [29]:
def separate_data(sample_size):
    indices = np.arange(sample_size)
    np.random.shuffle(indices)
    i1 = len(indices) // 3
    i2 = len(indices) // 3 * 2
    

    train_idx = indices[:i1]
    val_idx = indices[i1:i2]
    test_idx = indices[i2:]
    
    
    X_train = np.array(make_sample(train_idx))
    Y_train = np.array(Y.iloc[train_idx])
    
    X_val = np.array(make_sample(val_idx))
    Y_val = np.array(Y.iloc[val_idx])
    
    X_test = np.array(make_sample(test_idx))
    Y_test = np.array(Y.iloc[test_idx])
    return (X_train, Y_train, X_val, Y_val, X_test, Y_test)

In [30]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = separate_data(1000)

In [39]:
model = Sequential()
model.add(LSTM(256, input_shape=(11, 1500)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))


In [40]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               1799168   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               25700     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 1,824,969
Trainable params: 1,824,969
Non-trainable params: 0
_________________________________________________________________


In [41]:
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=128, validation_data=(X_val, Y_val))

Epoch 1/20
3/3 [==============================] - 1s 281ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss: 0.0000e+00 - val_accuracy: 0.0300
Epoch 2/20
3/3 [==============================] - 0s 153ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss: 0.0000e+00 - val_accuracy: 0.0300
Epoch 3/20
3/3 [==============================] - 0s 159ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss: 0.0000e+00 - val_accuracy: 0.0300
Epoch 4/20
3/3 [==============================] - 0s 155ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss: 0.0000e+00 - val_accuracy: 0.0300
Epoch 5/20
3/3 [==============================] - 0s 154ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss: 0.0000e+00 - val_accuracy: 0.0300
Epoch 6/20
3/3 [==============================] - 0s 166ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss: 0.0000e+00 - val_accuracy: 0.0300
Epoch 7/20
3/3 [==============================] - 1s 177ms/step - loss: 0.0000e+00 - accuracy: 0.0541 - val_loss

KeyboardInterrupt: 

In [37]:
from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(X_test)
accuracy_score(Y_test, test_preds)

0.038922155688622756